# Topic modelling - с помощью Gensim

In [1]:
!pip install pandas
!pip install gensim
!pip install nltk
!pip install pymorphy2
!pip install tqdm

import pandas as pd

import numpy as np

In [2]:
import nltk

nltk.download('stopwords')

from nltk.corpus import stopwords

from gensim.utils import simple_preprocess

from nltk.corpus import stopwords

import pymorphy2

from tqdm import tqdm

[nltk_data] Downloading package stopwords to /home/mitya/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Вначале - датасет. Это собранные короткие предложения с википедии, которые можно найти по ссылке:
https://wortschatz.uni-leipzig.de/en/download/Russian

Далее - предобработка. Возьмем наши предложения, разобъем на слова, удалим стоп-слова, лемматизируем остальные - приведем их к нормальной форме с помощью pymorphy2. 

In [3]:
sentences = open('rus_wikipedia_2021_1M-sentences.txt', 'r')
sentences_list = []
stop_words = stopwords.words('russian')
stop_words.extend(stopwords.words('english'))
morph = pymorphy2.MorphAnalyzer()
for i in tqdm(range(100000)):
    line = simple_preprocess(sentences.readline())
    l = []
    for word in line:
        if word not in stop_words:
            try:
                norm = morph.parse(word)[0].normal_form
                l.append(norm)
            except:
                l.append(word)
    sentences_list.append(l)

100%|██████████████████████████████████| 100000/100000 [02:51<00:00, 582.03it/s]


In [163]:
sentences_list[:5]

[['проект',
  'концепция',
  'свободный',
  'экономический',
  'зона',
  'одесса',
  'этап',
  'интеграция',
  'украина',
  'европейский',
  'союз'],
 ['декабрь', 'год'],
 ['дислокация',
  'войсковой',
  'часть',
  'штаб',
  'управление',
  'учреждение',
  'заведение',
  'рабочий',
  'крестьянский',
  'красный',
  'армия',
  'состояние',
  'июль',
  'год'],
 ['июль', 'забить', 'первый', 'мяч', 'клуб', 'поразить', 'ворота'],
 ['год',
  'уволить',
  'запас',
  'возраст',
  'формулировка',
  'период',
  'пребывание',
  'заключение',
  'ссылка',
  'засчитываться',
  'счёт',
  'пребывание',
  'служба',
  'кадр',
  'армия']]

Далее - создадим биграммы. Таким образом, Российская Федерация, Премьер-Лига и т.д. должны в итоге выглядеть как российский_федерация, премьер_лига, ...

In [5]:
from gensim.models import Phrases


bigram = Phrases(sentences_list, min_count=10, threshold=20)

In [164]:
list(bigram[sentences_list])[:5]

[['проект',
  'концепция',
  'свободный',
  'экономический',
  'зона',
  'одесса',
  'этап',
  'интеграция',
  'украина',
  'европейский',
  'союз'],
 ['декабрь', 'год'],
 ['дислокация',
  'войсковой',
  'часть',
  'штаб',
  'управление',
  'учреждение',
  'заведение',
  'рабочий_крестьянский',
  'красный_армия',
  'состояние',
  'июль',
  'год'],
 ['июль', 'забить', 'первый', 'мяч', 'клуб', 'поразить', 'ворота'],
 ['год',
  'уволить_запас',
  'возраст',
  'формулировка',
  'период',
  'пребывание',
  'заключение',
  'ссылка',
  'засчитываться',
  'счёт',
  'пребывание',
  'служба',
  'кадр',
  'армия']]

Далее - обучим Word2Vec. Подберем оптимальные параметры, ориентируясь на то, какие слова по идее должны стать синонимами. После этого обучим Word2Vec с оптимальными параметрами.

In [136]:
from gensim.models import Word2Vec


min_count_selection = [1, 3, 5, 7, 9]
window_selection = [3, 5]
vector_size_selection = [60, 70, 80]
grouped_params = [[min_count, window, vector_size] for min_count in min_count_selection 
                  for window in window_selection 
                  for vector_size in vector_size_selection]
synonyms = [('российский_федерация', 'рф'), ('машина', 'авто'), 
            ('университет', 'колледж'), ('эсминец', 'линкор'),
            ('село', 'деревня'), ('ссср', 'советский_союз')]
for group in grouped_params:
    cnt_syns = 0
    syn_diffs = 0.0
    model = Word2Vec(bigram[sentences_list], min_count=group[0], 
                     epochs=20, window=group[1], vector_size=group[2])
    print('min_count is: ' + str(group[0]) + ' and '
          'window is: ' + str(group[1]) + ' and vector_size is: ' + str(group[2]))
    closest_first = model.wv.most_similar(pair[0])
    closest_last = model.wv.most_similar(pair[1])
    for pair in synonyms:
        if model.wv.most_similar(pair[0])[0][0] == pair[1]:
            if model.wv.most_similar(pair[1])[0][0] == pair[0]:
                print(str(pair[0]) + ' and ' + pair[1] + ' are counted as synomyms')
                sum_diffs_first = model.wv.most_similar(pair[0])[0][1] - model.wv.most_similar(pair[0])[1][1]
                print('sum of differences from closest word to ' + pair[0] + ' is: ' + str(sum_diffs_first))
                sum_diffs_second = model.wv.most_similar(pair[1])[0][1] - model.wv.most_similar(pair[1])[1][1]
                print('sum of differences from closest word to ' + pair[1] + ' is: ' + str(sum_diffs_second))
                cnt_syns += 1
                syn_diffs += sum_diffs_first + sum_diffs_second

                
    print('number of found synonyms is ' + str(cnt_syns) + ' and total step is ' + str(syn_diffs))
    print(syn_diffs)
    print('#============================================================#')

min_count is: 1 and window is: 3 and vector_size is: 60
село and деревня are counted as synomyms
sum of differences from closest word to село is: 0.01161336898803711
sum of differences from closest word to деревня is: 0.04401487112045288
number of found synonyms is 1 and total step is 0.05562824010848999
0.05562824010848999
#============================================================#
min_count is: 1 and window is: 3 and vector_size is: 70
село and деревня are counted as synomyms
sum of differences from closest word to село is: 0.017843902111053467
sum of differences from closest word to деревня is: 0.0186956524848938
number of found synonyms is 1 and total step is 0.036539554595947266
0.036539554595947266
#============================================================#
min_count is: 1 and window is: 3 and vector_size is: 80
село and деревня are counted as synomyms
sum of differences from closest word to село is: 0.02577030658721924
sum of differences from closest word to деревня is: 0.

min_count is: 5 and window is: 5 and vector_size is: 60
университет and колледж are counted as synomyms
sum of differences from closest word to университет is: 0.0709492564201355
sum of differences from closest word to колледж is: 0.03701364994049072
село and деревня are counted as synomyms
sum of differences from closest word to село is: 0.029790878295898438
sum of differences from closest word to деревня is: 0.03038156032562256
number of found synonyms is 2 and total step is 0.16813534498214722
0.16813534498214722
#============================================================#
min_count is: 5 and window is: 5 and vector_size is: 70
российский_федерация and рф are counted as synomyms
sum of differences from closest word to российский_федерация is: 0.0014309287071228027
sum of differences from closest word to рф is: 0.031622469425201416
университет and колледж are counted as synomyms
sum of differences from closest word to университет is: 0.05161166191101074
sum of differences from clos

min_count is: 9 and window is: 5 and vector_size is: 60
российский_федерация and рф are counted as synomyms
sum of differences from closest word to российский_федерация is: 0.021044611930847168
sum of differences from closest word to рф is: 0.04711240530014038
село and деревня are counted as synomyms
sum of differences from closest word to село is: 0.010466694831848145
sum of differences from closest word to деревня is: 0.038625240325927734
number of found synonyms is 2 and total step is 0.11724895238876343
0.11724895238876343
#============================================================#
min_count is: 9 and window is: 5 and vector_size is: 70
российский_федерация and рф are counted as synomyms
sum of differences from closest word to российский_федерация is: 0.00393223762512207
sum of differences from closest word to рф is: 0.05048578977584839
университет and колледж are counted as synomyms
sum of differences from closest word to университет is: 0.0697488784790039
sum of differences fr

In [20]:
from gensim.models import Word2Vec


model = Word2Vec(bigram[sentences_list], min_count=3, 
                     epochs=20, window=5, vector_size=80)

In [21]:
model.wv.most_similar('рф')

[('розничный', 0.7888621091842651),
 ('доллар_сша', 0.7767943739891052),
 ('коммутация', 0.77588951587677),
 ('синтетический', 0.775061309337616),
 ('сотовый', 0.773492157459259),
 ('платёж', 0.7660769820213318),
 ('кофе', 0.7572059631347656),
 ('битум', 0.7567898631095886),
 ('выплавка', 0.7558321356773376),
 ('газа', 0.7531812191009521)]

In [23]:
model.wv.most_similar('российский_федерация')

[('рф', 0.8606249690055847),
 ('республика_беларусь', 0.8526559472084045),
 ('усср', 0.8525670170783997),
 ('президиум', 0.8207685947418213),
 ('совет_министр', 0.8127041459083557),
 ('рсфср', 0.8085163235664368),
 ('почётный_грамота', 0.7993912100791931),
 ('финансы', 0.779633641242981),
 ('присуждение', 0.7791944146156311),
 ('юстиция', 0.7762808203697205)]

Библиотека Faiss - уступает самому Gensim в качестве и скорости определения косинусного сходства, поэтому используем Gensim.

In [39]:
faiss_code = '''!pip install faiss-cpu

from faiss import IndexFlatIP

def standard_scale(list_coordinates):
    summ = 0
    for num in list_coordinates:
        summ += num**2
    return list_coordinates/summ**(1/2)


list_preprocessed = list(bigram[sentences_list])
count_dict = {}
for i in range(len(list_preprocessed)):
    for j in range(len(list_preprocessed[i])):
        word = list_preprocessed[i][j]
        count_dict[word] = count_dict.get(word, 0) + 1
index = IndexFlatIP(80)
for word in count_dict.keys():
    index.add(np.array([standard_scale(model.wv[word])]))'''

Для каждой пары синонимов найдем более популярное слово, и заменим второе из пары на него, во всех текстах.

In [42]:
replace_dict = {}
for i in tqdm(range(len(list_preprocessed))):
    for j in range(len(list_preprocessed[i])):
        word = list_preprocessed[i][j]
        if replace_dict.get(word, None) != None:
            list_preprocessed[i][j] = replace_dict.get(word)
        else:
            #D, I = index.search(standard_scale(model.wv[[word]]), 2) 
            #counterpart = list(count_dict.keys())[I.squeeze()[1]]
            #D, I = index.search(standard_scale(model.wv[[counterpart]]), 2) 
            #if word == list(count_dict.keys())[I.squeeze()[1]]:
            try:
                counterpart = model.wv.most_similar(word, topn=1)[0][0]
                if word == model.wv.most_similar(counterpart, topn=1)[0][0]:
                    if count_dict[counterpart] >= count_dict[word]:
                        replace_dict[word] = counterpart
                        list_preprocessed[i][j] = counterpart
                    else:
                        replace_dict[counterpart] = word
            except:
                pass

100%|██████████████████████████████████| 100000/100000 [13:32<00:00, 123.06it/s]


In [43]:
replace_dict

{'экономика': 'экономический',
 'заведение': 'учебный_заведение',
 'июль': 'апрель',
 'забить': 'гол',
 'забросить': 'ворота',
 'уволить_запас': 'демобилизовать',
 'взрослый': 'возраст',
 'кадр': 'подготовка',
 'войско': 'армия',
 'существенный': 'значительный',
 'суметь': 'смочь',
 'www': 'https',
 'премьера': 'дебют',
 'кхл': 'премьер_лига',
 'синий': 'жёлтый',
 'турне': 'тур',
 'кубок': 'чемпионат',
 'играть': 'выступать',
 'преимущественно': 'основное',
 'первый_дивизион': 'дубль',
 'кавказ': 'сибирь',
 'приблизительно': 'полтора',
 'три': 'два',
 'получать': 'получить',
 'выходить': 'выйти',
 'четвертьфинал': 'полуфинал',
 'уефа': 'кубок_англия',
 'основать': 'создать',
 'школа_пилот': 'военный_авиационный',
 'сей_пора': 'настоящий_время',
 'выпускаться': 'выпускать',
 'газета': 'журнал',
 'шум': 'проба',
 'видимый': 'обычно',
 'генетический': 'анализ',
 'внешне': 'следующий_образ',
 'сформулировать': 'заключаться',
 'абстрактный': 'смысл',
 'должный': 'тот',
 'фирма': 'компания',

In [47]:
list_preprocessed

[['проект',
  'концепция',
  'свободный',
  'экономический',
  'зона',
  'одесса',
  'этап',
  'интеграция',
  'украина',
  'европейский',
  'союз'],
 ['декабрь', 'год'],
 ['дислокация',
  'войсковой',
  'часть',
  'штаб',
  'управление',
  'учреждение',
  'учебный_заведение',
  'рабочий_крестьянский',
  'красный_армия',
  'состояние',
  'апрель',
  'год'],
 ['апрель', 'гол', 'первый', 'мяч', 'клуб', 'поразить', 'ворота'],
 ['год',
  'демобилизовать',
  'возраст',
  'формулировка',
  'период',
  'пребывание',
  'заключение',
  'ссылка',
  'засчитываться',
  'счёт',
  'пребывание',
  'служба',
  'подготовка',
  'армия'],
 ['значительный',
  'вклад',
  'образование',
  'воспитание',
  'подрастать',
  'поколение',
  'вручить',
  'значок'],
 ['балл',
  'ошибка',
  'компьютер',
  'смочь',
  'обеспечить',
  'отрыв',
  'соперник',
  'выиграть',
  'высокий_лига',
  'https',
  'web'],
 ['затем', 'счёт', 'дом', 'одержать', 'верх'],
 ['многие', 'болельщик', 'потребовать', 'отставка'],
 ['состоять

Построим модель lda, выделим с ее помощью 15 тем.

In [52]:
import gensim

import gensim.corpora as corpora


dict_articles = corpora.Dictionary(list_preprocessed)
corpus = [dict_articles.doc2bow(article) for article in list_preprocessed]
lda = gensim.models.ldamodel.LdaModel(corpus=corpus, id2word=dict_articles, num_topics=15, 
                                      update_every=1, chunksize=100, passes=10, per_word_topics=True)

In [121]:
lda.print_topics()

[(0,
  '0.128*"свой" + 0.101*"состав" + 0.092*"чемпионат" + 0.084*"выпустить" + 0.059*"два" + 0.040*"украина" + 0.029*"дебютировать" + 0.013*"заявить" + 0.013*"победить" + 0.012*"победа"'),
 (1,
  '0.145*"компания" + 0.090*"место" + 0.065*"журнал" + 0.037*"российский" + 0.032*"лейбл" + 0.030*"собственный" + 0.026*"приобрести" + 0.024*"крупный" + 0.023*"тот" + 0.022*"другой"'),
 (2,
  '0.091*"страна" + 0.087*"впервые" + 0.073*"последний" + 0.065*"игрок" + 0.047*"территория" + 0.036*"смочь" + 0.019*"полный" + 0.014*"юг" + 0.012*"пытаться" + 0.009*"север"'),
 (3,
  '0.037*"область" + 0.037*"район" + 0.037*"житель" + 0.030*"составлять" + 0.029*"часть" + 0.029*"право" + 0.026*"региональный" + 0.024*"уровень" + 0.024*"больший" + 0.024*"северный"'),
 (4,
  '0.155*"клуб" + 0.072*"работа" + 0.049*"её" + 0.036*"национальный" + 0.032*"сыграть" + 0.027*"имя" + 0.025*"записать" + 0.023*"история" + 0.022*"возглавить" + 0.022*"начало"'),
 (5,
  '0.069*"центр" + 0.061*"предприятие" + 0.040*"село" + 0.

Проверим модель в деле. Посмотрим, к каким темам она отнесет это предложение.

In [149]:
list_preprocessed[100]

['октябрь', 'костромской', 'район', 'корова', 'костромской', 'порода']

In [146]:
lda.get_document_topics(dict_articles.doc2bow(list_preprocessed[100]))

[(0, 0.022226706),
 (1, 0.022226706),
 (2, 0.022226706),
 (3, 0.35564786),
 (4, 0.022226706),
 (5, 0.022226706),
 (6, 0.35540497),
 (7, 0.022226706),
 (8, 0.022226706),
 (9, 0.022226706),
 (10, 0.022226706),
 (11, 0.022226706),
 (12, 0.022226706),
 (13, 0.022226706),
 (14, 0.022226706)]

In [152]:
print(lda.show_topics(15)[3])
print(lda.show_topics(15)[6])

(3, '0.037*"область" + 0.037*"район" + 0.037*"житель" + 0.030*"составлять" + 0.029*"часть" + 0.029*"право" + 0.026*"региональный" + 0.024*"уровень" + 0.024*"больший" + 0.024*"северный"')
(6, '0.679*"год" + 0.024*"перейти" + 0.024*"команда" + 0.024*"группа" + 0.013*"вернуться" + 0.012*"вновь" + 0.010*"чемпионат_мир" + 0.010*"являться" + 0.008*"песня" + 0.007*"сборная"')


Заспарсим пару страниц яндекс новостей, и проверим на них.

In [153]:
!pip install requests
!pip install bs4
!pip install fake_useragent

import requests

from fake_useragent import UserAgent

from bs4 import BeautifulSoup as bs


url = 'https://yandex.ru/news'
response = requests.get(url, headers={'User-Agent': UserAgent().chrome})
soup = bs(response.text, 'html')

descriptions = soup.find_all('div', {'class': 'mg-card__text'})
links = [description.a.get('href') for description in descriptions]
texts = []
for i in range(3):
    import time
    
    
    time.sleep(3)  
    
    link = links[i]
    response = requests.get(link)
    soup = bs(response.text, 'html')
    lines = soup.find_all('div', {'class': 'mg-snippet__content'})
    text = ''
    for line in lines:
        try:
            texts.append(line.div.span.text)
        except:
            texts.append(line.div.text)

In [155]:
texts_preprocessed = []
for text in (texts):
    line = simple_preprocess(text)
    l = []
    for word in line:
        if word not in stop_words:
            try:
                norm = morph.parse(word)[0].normal_form
                l.append(norm)
            except:
                l.append(word)
    texts_preprocessed.append(l)
    
texts_nicely_done = list(bigram[texts_preprocessed])
for i in range(len(texts_nicely_done)):
    for j in range(len(texts_nicely_done[i])):
        word = texts_nicely_done[i][j]
        if replace_dict.get(word, None) != None:
            texts_nicely_done[i][j] = replace_dict.get(word)

In [161]:
texts[0]

' Группа депутатов от «Единой России» внесла в нижнюю палату парламента законопроект, который определяет условия и механизм введения внешней администрации на российских подразделениях иностранных компаний, покинувших Россию. Предполагается, что решение о введении внешнего управления будет приниматься межведомственной комиссией при Минэкономразвития. Переход управления от иностранных собственников к внешнему управлению будет назначаться только по решению суда. Иностранным собственникам сохранят возможность возобновить работу в России или продать пакет акций. В\xa0Госдуму внесли законопроект об управлении имуществом ушедших компаний Внесен законопроект о\xa0внешнем управлении иностранными компаниями, которые объявили об уходе с\xa0российского рынка В\xa0Госдуму внесли законопроект о\xa0внешнем управлении иностранным бизнесом Единая Россия внесла в\xa0Госдуму законопроект о\xa0внешнем управлении иностранными компаниями, которые объявили об уходе с\xa0российского рынка ЕР внесла в\xa0Госду

In [158]:
lda.get_document_topics(dict_articles.doc2bow(texts_nicely_done[0]))

[(1, 0.2517039),
 (3, 0.026609164),
 (4, 0.051665258),
 (5, 0.02666249),
 (6, 0.05164736),
 (8, 0.2517225),
 (9, 0.026663115),
 (10, 0.05166066),
 (12, 0.15167378),
 (13, 0.10165819)]

In [160]:
print(lda.show_topics(15)[1])
print(lda.show_topics(15)[8])
print(lda.show_topics(15)[12])
print(lda.show_topics(15)[13])

(1, '0.145*"компания" + 0.090*"место" + 0.065*"журнал" + 0.037*"российский" + 0.032*"лейбл" + 0.030*"собственный" + 0.026*"приобрести" + 0.024*"крупный" + 0.023*"тот" + 0.022*"другой"')
(8, '0.200*"новый" + 0.049*"управление" + 0.031*"контракт" + 0.029*"установить" + 0.029*"затем" + 0.023*"подписать" + 0.023*"художник" + 0.020*"срок" + 0.020*"материал" + 0.015*"произведение"')
(12, '0.144*"стать" + 0.058*"россия" + 0.035*"название" + 0.032*"сша" + 0.031*"книга" + 0.022*"игра" + 0.021*"третий" + 0.020*"лига" + 0.018*"государственный" + 0.018*"издательство"')
(13, '0.103*"который" + 0.052*"фильм" + 0.037*"альбом" + 0.030*"сериал" + 0.029*"также" + 0.027*"сезон" + 0.026*"работать" + 0.025*"начать" + 0.023*"появиться" + 0.020*"участвовать"')
